In [96]:
import pandas as pd
import numpy as np

List of dfs needed:

1. tyre_data_new_df x
2. circuits_df x
3. drivers_df x
4. lap_times_df
5. pit_stops_df x
6. races_with_dummies (included weather data in dummie variables) x
7. results_df x

Optional:

1. constructor_results_df
2. constructors_df
3. status_df

In [97]:
circuits_df = pd.read_csv("data/circuits.csv")
drivers_df = pd.read_csv("data/drivers.csv")
pit_stops_df = pd.read_csv("data/pit_stops.csv")
races_with_dummies_df = pd.read_csv("data/races_with_dummies.csv")
results_df = pd.read_csv("data/results.csv")
tyre_data_ID_df = pd.read_csv("data/tyre_data_drRef.csv")
lap_times_df = pd.read_csv("data/lap_times.csv")




In [98]:
tyre_data_ID_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13302 entries, 0 to 13301
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  13302 non-null  int64  
 1   Driver      13297 non-null  object 
 2   GP          13302 non-null  object 
 3   Tyres       13302 non-null  object 
 4   From        13302 non-null  int64  
 5   To          13302 non-null  int64  
 6   #Laps       13302 non-null  int64  
 7   year        13302 non-null  int64  
 8   location    11353 non-null  object 
 9   raceId      13302 non-null  int64  
 10  driverId    13164 non-null  float64
 11  driverRef   13297 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 1.2+ MB


#### Cleaning

__Tyre_data_Id_df__

In [99]:
tyres_mapping = {
    'Hiper Blando Usado': 'Hyper Soft',
    'Hiper Blando Nuevo': 'Hyper Soft',
    'Lluvia Extrema Usado': 'Full Wet'
}

# Use the replace() method to update the values in the "tyres" column
tyre_data_ID_df['Tyres'] = tyre_data_ID_df['Tyres'].replace(tyres_mapping)


In [100]:
tyre_data_ID_df['Tyres'] = tyre_data_ID_df['Tyres'].str.replace(' Used', '').str.replace(' New', '')

# Print the DataFrame to verify the changes
print(tyre_data_ID_df)

       Unnamed: 0            Driver           GP         Tyres  From  To   
0               0     Jenson Button    Australia          Soft     1  16  \
1               1     Jenson Button    Australia        Medium    17  36   
2               2     Jenson Button    Australia        Medium    37  58   
3               3  Sebastian Vettel    Australia          Soft     1  16   
4               4  Sebastian Vettel    Australia          Soft    17  37   
...           ...               ...          ...           ...   ...  ..   
13297       13297       Zhou Guanyu  Netherlands          Soft    38  60   
13298       13298    Logan Sargeant  Netherlands          Soft     1  14   
13299       13299    Max Verstappen  Netherlands  Intermediate    65  72   
13300       13300      Pierre Gasly  Netherlands          Soft    47  60   
13301       13301   Nico Hulkenberg  Netherlands        Medium    17  48   

       #Laps  year   location  raceId  driverId   driverRef  
0         16  2012   Adel

In [101]:
tyre_data_ID_df.Tyres.unique()

array(['Soft', 'Medium', 'Intermediate', 'Wet', 'Hard', 'Full Wet',
       'Supersoft', 'Ultrasoft', 'Hyper Soft'], dtype=object)

In [102]:
tyre_data_ID_df = pd.DataFrame(data)

# Split the 'Tyres' column
tyre_data_ID_df['Tyres'] = tyre_data_ID_df['Tyres'].str.split(', ')

# Split the 'From' column
tyre_data_ID_df['From'] = tyre_data_ID_df['From'].str.split(', ')

# Split the 'To' column
tyre_data_ID_df['To'] = tyre_data_ID_df['To'].str.split(', ')

# Expand the columns without specifying 'Tyres' since it no longer exists
tyres_columns = tyre_data_ID_df['Tyres'].apply(pd.Series)
from_columns = tyre_data_ID_df['From'].apply(pd.Series)
to_columns = tyre_data_ID_df['To'].apply(pd.Series)

# Rename the expanded columns
tyres_columns.columns = [f'Tyres_{i+1}' for i in range(tyres_columns.shape[1])]
from_columns.columns = [f'From_{i+1}' for i in range(from_columns.shape[1])]
to_columns.columns = [f'To_{i+1}' for i in range(to_columns.shape[1])]

# Concatenate the expanded columns with the original DataFrame
tyre_data_ID_df = pd.concat([tyre_data_ID_df, tyres_columns, from_columns, to_columns], axis=1)

# Drop the original 'Tyres,' 'From,' and 'To' columns
tyre_data_ID_df = tyre_data_ID_df.drop(['Tyres', 'From', 'To'], axis=1)

# Print the resulting DataFrame
print(tyre_data_ID_df)

          Driver         GP  year  location  raceId  driverId driverRef   
0  Jenson Button  Australia  2012  Adelaide     860      18.0    button  \

   #Laps    Tyres_1     Tyres_2     Tyres_3 From_1 From_2 From_3 To_1 To_2   
0     16  Soft Used  Medium New  Medium New      1     17     37   16   36  \

  To_3  
0   58  


In [103]:
tyre_data_ID_df.head(40)

,Driver,GP,year,location,raceId,driverId,driverRef,#Laps,Tyres_1,Tyres_2,Tyres_3,From_1,From_2,From_3,To_1,To_2,To_3
0,Jenson Button,Australia,2012,Adelaide,860,18.0,button,16,Soft Used,Medium New,Medium New,1,17,37,16,36,58


__Pit_Stops_df__

In [43]:
pit_stops_df.head()

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842


In [44]:
pit_stops_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9634 entries, 0 to 9633
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   raceId        9634 non-null   int64 
 1   driverId      9634 non-null   int64 
 2   stop          9634 non-null   int64 
 3   lap           9634 non-null   int64 
 4   time          9634 non-null   object
 5   duration      9634 non-null   object
 6   milliseconds  9634 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 527.0+ KB


In [45]:
new_pit_stops_df = pit_stops_df.drop(columns=['duration'])

In [46]:
new_pit_stops_df.head(40)

,raceId,driverId,stop,lap,time,milliseconds
0,841,153,1,1,17:05:23,26898
1,841,30,1,1,17:05:52,25021
2,841,17,1,11,17:20:48,23426
3,841,4,1,12,17:22:34,23251
4,841,13,1,13,17:24:10,23842
5,841,22,1,13,17:24:29,23643
6,841,20,1,14,17:25:17,22603
7,841,814,1,14,17:26:03,24863
8,841,816,1,14,17:26:50,25259
9,841,67,1,15,17:27:34,25342


In [47]:
import pandas as pd

# Assuming you already have new_pit_stops_df

# Group by "raceId" and "driverId" and aggregate the other columns
aggregation_functions = {
    'stop': 'sum',
    'lap': 'sum',
    'time': 'first',  # You can choose an aggregation method for time
    'milliseconds': 'sum'
}

# Apply the aggregation functions
result_df = new_pit_stops_df.groupby(['raceId', 'driverId']).agg(aggregation_functions).reset_index()

# Print the resulting DataFrame
print(result_df)


      raceId  driverId  stop  lap      time  milliseconds
0        841         1     3   52  17:28:24         46426
1        841         2     3   45  17:27:41         48092
2        841         3     1   16  17:29:00         23716
3        841         4     6   81  17:22:34         72165
4        841         5     1   17  17:31:11         24865
...      ...       ...   ...  ...       ...           ...
4696    1096       848     3   47  17:21:53         43591
4697    1096       849     3   53  17:26:44         47597
4698    1096       852     3   52  17:24:50         43853
4699    1096       854     3   48  17:24:59         45806
4700    1096       855     3   59  17:26:28         45594

[4701 rows x 6 columns]


In [48]:
result_df.head(20)

,raceId,driverId,stop,lap,time,milliseconds
0,841,1,3,52,17:28:24,46426
1,841,2,3,45,17:27:41,48092
2,841,3,1,16,17:29:00,23716
3,841,4,6,81,17:22:34,72165
4,841,5,1,17,17:31:11,24865
5,841,10,1,18,17:33:02,23792
6,841,13,6,92,17:24:10,72437
7,841,15,3,53,17:29:49,49747
8,841,16,3,53,17:29:08,49849
9,841,17,6,78,17:20:48,72176


In [49]:
races_with_dummies_df.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,...,quali_time,sprint_date,sprint_time,weather_category,weather_cloudy,weather_cold,weather_dry,weather_unknown,weather_warm,weather_wet
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,...,\N,\N,\N,weather_warm,False,False,False,False,True,False
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,...,\N,\N,\N,weather_dry,False,False,True,False,False,False
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,...,\N,\N,\N,weather_wet,False,False,False,False,False,True
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,...,\N,\N,\N,weather_warm,False,False,False,False,True,False
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,...,\N,\N,\N,weather_warm,False,False,False,False,True,False


In [50]:
import pandas as pd

# Assuming you have read the CSV file into a DataFrame named 'df'
# Replace 'your_file.csv' with the actual path to your CSV file
# df = pd.read_csv('your_file.csv')

# Group by "raceId" and "driverId" (in this case, "Jenson Button")
grouped = tyre_data_ID_df.groupby(['raceId', 'driverId'])

# Define an aggregation function for each column (you can choose how to aggregate)
aggregation = {
    'Driver': 'first',   # Take the first driver's name
    'GP': 'first',       # Take the first GP value
    'Tyres': 'first',    # Take the first Tyres value
    'From': 'min',       # Take the minimum value of 'From'
    'To': 'max',         # Take the maximum value of 'To'
    '#Laps': 'sum',      # Sum of '#Laps'
    'year': 'first',     # Take the first year
    'location': 'first', # Take the first location
    'driverRef': 'first' # Take the first driverRef
}

# Apply the aggregation
transformed_df = grouped.agg(aggregation).reset_index()

# Print the resulting DataFrame
print(transformed_df)


      raceId  driverId             Driver           GP         Tyres  From   
0        860       1.0     Lewis Hamilton    Australia          Soft     1  \
1        860       3.0       Nico Rosberg    Australia          Soft     1   
2        860       4.0    Fernando Alonso    Australia        Medium     1   
3        860       5.0  Heikki Kovalainen    Australia          Soft     1   
4        860       8.0     Kimi Räikkönen    Australia          Soft     1   
...      ...       ...                ...          ...           ...   ...   
4533    1111     847.0     George Russell  Netherlands          Soft     1   
4534    1111     848.0    Alexander Albon  Netherlands          Soft     1   
4535    1111     852.0       Yuki Tsunoda  Netherlands          Soft     1   
4536    1111     857.0      Oscar Piastri  Netherlands  Intermediate     1   
4537    1111     858.0     Logan Sargeant  Netherlands          Soft     1   

      To  #Laps  year   location       driverRef  
0     58    

In [56]:
import pandas as pd
import numpy as np

# Assuming you have read the CSV file into a DataFrame named 'df'
# Replace 'your_file.csv' with the actual path to your CSV file
# df = pd.read_csv('your_file.csv')

# Filter rows for Jenson Button
jenson_button_df = tyre_data_ID_df[tyre_data_ID_df['Driver'] == 'Jenson Button']

# Create an empty list to store DataFrames
dfs_to_concat = []

# Iterate through unique raceId and driverId combinations
for race_id, driver_id in jenson_button_df[['raceId', 'driverId']].drop_duplicates().itertuples(index=False):
    race_driver_df = jenson_button_df[(jenson_button_df['raceId'] == race_id) & (jenson_button_df['driverId'] == driver_id)]
    
    # Group by 'GP' and concatenate the different tire types
    tire_types = ', '.join(race_driver_df['Tyres'])
    
    # Take the first value for other columns (assumes they are consistent for the same race and driver)
    first_values = race_driver_df.iloc[0]
    
    # Create a new DataFrame for the row
    new_row_df = pd.DataFrame({
        'Driver': [first_values['Driver']],
        'GP': [first_values['GP']],
        'Tyres': [tire_types],  # Concatenated tire types
        'From': [None],  # You can set this to None or adjust as needed
        'To': [None],  # You can set this to None or adjust as needed
        '#Laps': [first_values['#Laps']],
        'year': [first_values['year']],
        'location': [first_values['location']],
        'raceId': [race_id],
        'driverId': [driver_id],
        'driverRef': [first_values['driverRef']]
    })
    
    # Append the new row DataFrame to the list
    dfs_to_concat.append(new_row_df)

# Concatenate all the DataFrames in the list into one
transformed_df = pd.concat(dfs_to_concat, ignore_index=True)

# Print the resulting DataFrame
transformed_df.head()


,Driver,GP,Tyres,From,To,#Laps,year,location,raceId,driverId,driverRef
0,Jenson Button,Australia,"Soft, Medium, Medium",None,None,16,2012,Adelaide,860,18.0,button
1,Jenson Button,Malaysia,"Intermediate, Wet, Intermediate, Intermediate,...",None,None,4,2012,Kuala Lumpur,861,18.0,button
2,Jenson Button,China,"Soft, Soft, Medium, Medium",None,None,11,2012,Shanghai,862,18.0,button
3,Jenson Button,Bahrain,"Soft, Medium, Medium, Medium, Soft",None,None,8,2012,Sakhir,863,18.0,button
4,Jenson Button,Spain,"Soft, Hard, Soft, Hard",None,None,9,2012,Barcelona,864,18.0,button


In [57]:
import pandas as pd

# Assuming you have a DataFrame named 'transformed_df'
# Split the values in the 'Tyres' column by ',' and expand them into separate columns
tire_type_columns = transformed_df['Tyres'].str.split(', ', expand=True)

# Concatenate the tire type columns with the original DataFrame
transformed_df = pd.concat([transformed_df, tire_type_columns], axis=1)

# Drop the original 'Tyres' column
transformed_df = transformed_df.drop('Tyres', axis=1)

transformed_df.head()


,Driver,GP,From,To,#Laps,year,location,raceId,driverId,driverRef,0,1,2,3,4,5,6
0,Jenson Button,Australia,None,None,16,2012,Adelaide,860,18.0,button,Soft,Medium,Medium,None,None,None,None
1,Jenson Button,Malaysia,None,None,4,2012,Kuala Lumpur,861,18.0,button,Intermediate,Wet,Intermediate,Intermediate,Medium,None,None
2,Jenson Button,China,None,None,11,2012,Shanghai,862,18.0,button,Soft,Soft,Medium,Medium,None,None,None
3,Jenson Button,Bahrain,None,None,8,2012,Sakhir,863,18.0,button,Soft,Medium,Medium,Medium,Soft,None,None
4,Jenson Button,Spain,None,None,9,2012,Barcelona,864,18.0,button,Soft,Hard,Soft,Hard,None,None,None


In [60]:
import pandas as pd

# Sample DataFrame
data = {
    'From': ['1-16, 17-36, 37-58', '1-4, 5-13, 14-24, 25-39, 40-56', '1-11, 12-24, 25-39, 40-56', '1-8, 9-22, 23-37, 38-53, 54-55', '1-9, 10-25, 26-38, 39-66'],
    'To': [None, None, None, None, None],
    '#Laps': [16, 4, 11, 8, 9],
    'year': [2012, 2012, 2012, 2012, 2012],
    'location': ['Adelaide', 'Kuala Lumpur', 'Shanghai', 'Sakhir', 'Barcelona'],
    'raceId': [860, 861, 862, 863, 864],
    'driverId': [18.0, 18.0, 18.0, 18.0, 18.0],
    'driverRef': ['button', 'button', 'button', 'button', 'button']
}

df = pd.DataFrame(data)

# Add Driver and GP columns
df['Driver'] = 'Jenson Button'
df['GP'] = df['location']  # Assuming 'location' is the GP name

# Add Tyres column (you can replace 'Soft' with the actual tire type if needed)
df['Tyres'] = 'Soft'

# Rearrange the columns if necessary
df = df[['Driver', 'GP', 'Tyres', 'From', 'To', '#Laps', 'year', 'location', 'raceId', 'driverId', 'driverRef']]

# Print the resulting DataFrame
df.head()


,Driver,GP,Tyres,From,To,#Laps,year,location,raceId,driverId,driverRef
0,Jenson Button,Adelaide,Soft,"1-16, 17-36, 37-58",None,16,2012,Adelaide,860,18.0,button
1,Jenson Button,Kuala Lumpur,Soft,"1-4, 5-13, 14-24, 25-39, 40-56",None,4,2012,Kuala Lumpur,861,18.0,button
2,Jenson Button,Shanghai,Soft,"1-11, 12-24, 25-39, 40-56",None,11,2012,Shanghai,862,18.0,button
3,Jenson Button,Sakhir,Soft,"1-8, 9-22, 23-37, 38-53, 54-55",None,8,2012,Sakhir,863,18.0,button
4,Jenson Button,Barcelona,Soft,"1-9, 10-25, 26-38, 39-66",None,9,2012,Barcelona,864,18.0,button


In [64]:
import pandas as pd

# Sample DataFrame
data = {
    'Driver': ['Jenson Button', 'Jenson Button', 'Jenson Button'],
    'GP': ['Australia', 'Australia', 'Australia'],
    'Tyres': ['Soft Used', 'Medium New', 'Medium New'],
    'From': [1, 17, 37],
    'To': [16, 36, 58],
    '#Laps': [16, 20, 22],
    'year': [2012, 2012, 2012],
    'location': ['Adelaide', 'Adelaide', 'Adelaide'],
    'raceId': [860, 860, 860],
    'driverId': [18.0, 18.0, 18.0],
    'driverRef': ['button', 'button', 'button']
}

df = pd.DataFrame(data)

# Convert 'From' and 'To' columns to strings
df['From'] = df['From'].astype(str)
df['To'] = df['To'].astype(str)

# Aggregate duplicate rows by concatenating values
df = df.groupby(['Driver', 'GP', 'year', 'location', 'raceId', 'driverId', 'driverRef']).agg({
    'Tyres': ', '.join,
    'From': ', '.join,
    'To': ', '.join,
    '#Laps': 'first'  # Use 'first' to keep one value
}).reset_index()

# Print the resulting DataFrame
df.head()


,Driver,GP,year,location,raceId,driverId,driverRef,Tyres,From,To,#Laps
0,Jenson Button,Australia,2012,Adelaide,860,18.0,button,"Soft Used, Medium New, Medium New","1, 17, 37","16, 36, 58",16


In [66]:
import pandas as pd

# Sample DataFrame
data = {
    'Driver': ['Jenson Button'],
    'GP': ['Australia'],
    'year': [2012],
    'location': ['Adelaide'],
    'raceId': [860],
    'driverId': [18.0],
    'driverRef': ['button'],
    'Tyres': ['Soft Used, Medium New, Medium New'],
    'From': ['1, 17, 37'],
    'To': ['16, 36, 58'],
    '#Laps': [16]
}

df = pd.DataFrame(data)

# Split the 'Tyres' column
df['Tyres'] = df['Tyres'].str.split(', ')

# Split the 'From' column
df['From'] = df['From'].str.split(', ')

# Split the 'To' column
df['To'] = df['To'].str.split(', ')

# Print the resulting DataFrame
df.head()


,Driver,GP,year,location,raceId,driverId,driverRef,Tyres,From,To,#Laps
0,Jenson Button,Australia,2012,Adelaide,860,18.0,button,"[Soft Used, Medium New, Medium New]","[1, 17, 37]","[16, 36, 58]",16


In [71]:
import pandas as pd

# Sample DataFrame
data = {
    'Driver': ['Jenson Button'],
    'GP': ['Australia'],
    'year': [2012],
    'location': ['Adelaide'],
    'raceId': [860],
    'driverId': [18.0],
    'driverRef': ['button'],
    'Tyres': ['Soft Used, Medium New, Medium New'],
    'From': ['1, 17, 37'],
    'To': ['16, 36, 58'],
    '#Laps': [16]
}

df = pd.DataFrame(data)

# Split the 'Tyres' column
df['Tyres'] = df['Tyres'].str.split(', ')

# Split the 'From' column
df['From'] = df['From'].str.split(', ')

# Split the 'To' column
df['To'] = df['To'].str.split(', ')

# Expand the columns without specifying 'Tyres' since it no longer exists
tyres_columns = df['Tyres'].apply(pd.Series)
from_columns = df['From'].apply(pd.Series)
to_columns = df['To'].apply(pd.Series)

# Rename the expanded columns
tyres_columns.columns = [f'Tyres_{i+1}' for i in range(tyres_columns.shape[1])]
from_columns.columns = [f'From_{i+1}' for i in range(from_columns.shape[1])]
to_columns.columns = [f'To_{i+1}' for i in range(to_columns.shape[1])]

# Concatenate the expanded columns with the original DataFrame
df = pd.concat([df, tyres_columns, from_columns, to_columns], axis=1)

# Drop the original 'Tyres,' 'From,' and 'To' columns
df = df.drop(['Tyres', 'From', 'To'], axis=1)

# Print the resulting DataFrame
print(df)


          Driver         GP  year  location  raceId  driverId driverRef   
0  Jenson Button  Australia  2012  Adelaide     860      18.0    button  \

   #Laps    Tyres_1     Tyres_2     Tyres_3 From_1 From_2 From_3 To_1 To_2   
0     16  Soft Used  Medium New  Medium New      1     17     37   16   36  \

  To_3  
0   58  


In [72]:
df.head()

,Driver,GP,year,location,raceId,driverId,driverRef,#Laps,Tyres_1,Tyres_2,Tyres_3,From_1,From_2,From_3,To_1,To_2,To_3
0,Jenson Button,Australia,2012,Adelaide,860,18.0,button,16,Soft Used,Medium New,Medium New,1,17,37,16,36,58


In [ ]:
df = pd.DataFrame(data)

# Split the 'Tyres' column
df['Tyres'] = df['Tyres'].str.split(', ')

# Split the 'From' column
df['From'] = df['From'].str.split(', ')

# Split the 'To' column
df['To'] = df['To'].str.split(', ')

# Expand the columns without specifying 'Tyres' since it no longer exists
tyres_columns = df['Tyres'].apply(pd.Series)
from_columns = df['From'].apply(pd.Series)
to_columns = df['To'].apply(pd.Series)

# Rename the expanded columns
tyres_columns.columns = [f'Tyres_{i+1}' for i in range(tyres_columns.shape[1])]
from_columns.columns = [f'From_{i+1}' for i in range(from_columns.shape[1])]
to_columns.columns = [f'To_{i+1}' for i in range(to_columns.shape[1])]

# Concatenate the expanded columns with the original DataFrame
df = pd.concat([df, tyres_columns, from_columns, to_columns], axis=1)

# Drop the original 'Tyres,' 'From,' and 'To' columns
df = df.drop(['Tyres', 'From', 'To'], axis=1)

# Print the resulting DataFrame
print(df)

In [ ]:
tyre_data_ID_df

In [84]:
tyre_data_ID_df = pd.DataFrame(data)

# Split the 'Tyres' column
tyre_data_ID_df['Tyres'] = tyre_data_ID_df['Tyres'].str.split(', ')

# Split the 'From' column
tyre_data_ID_df['From'] = tyre_data_ID_df['From'].str.split(', ')

# Split the 'To' column
tyre_data_ID_df['To'] = tyre_data_ID_df['To'].str.split(', ')

# Expand the columns without specifying 'Tyres' since it no longer exists
tyres_columns = tyre_data_ID_df['Tyres'].apply(pd.Series)
from_columns = tyre_data_ID_df['From'].apply(pd.Series)
to_columns = tyre_data_ID_df['To'].apply(pd.Series)

# Rename the expanded columns
tyres_columns.columns = [f'Tyres_{i+1}' for i in range(tyres_columns.shape[1])]
from_columns.columns = [f'From_{i+1}' for i in range(from_columns.shape[1])]
to_columns.columns = [f'To_{i+1}' for i in range(to_columns.shape[1])]

# Concatenate the expanded columns with the original DataFrame
tyre_data_ID_df = pd.concat([tyre_data_ID_df, tyres_columns, from_columns, to_columns], axis=1)

# Drop the original 'Tyres,' 'From,' and 'To' columns
tyre_data_ID_df = tyre_data_ID_df.drop(['Tyres', 'From', 'To'], axis=1)

# Print the resulting DataFrame
print(tyre_data_ID_df)


          Driver         GP  year  location  raceId  driverId driverRef   
0  Jenson Button  Australia  2012  Adelaide     860      18.0    button  \

   #Laps    Tyres_1     Tyres_2     Tyres_3 From_1 From_2 From_3 To_1 To_2   
0     16  Soft Used  Medium New  Medium New      1     17     37   16   36  \

  To_3  
0   58  


In [87]:
tyre_data_ID_df.head(50)

,Driver,GP,year,location,raceId,driverId,driverRef,#Laps,Tyres_1,Tyres_2,Tyres_3,From_1,From_2,From_3,To_1,To_2,To_3
0,Jenson Button,Australia,2012,Adelaide,860,18.0,button,16,Soft Used,Medium New,Medium New,1,17,37,16,36,58
